In [ ]:
import pickle
import numpy as np

# Load dataset
with open('D:/AIT/deepmimodata/data/processed/test_dataset.pkl', 'rb') as f:
    data = pickle.load(f)

# Access training data for urban_60 velocity
urban_data = data['velocity_data']['urban_60']
X_train = urban_data['splits']['X_train']  # (N, T, Nr, Nt, Nk)
Y_train = urban_data['splits']['Y_train']  # (N, Q, Nr, Nt, Nk)

print(f"Training sequences: {X_train.shape}")
print(f"Velocity: {urban_data['metadata']['velocity_kmh']} km/h")
print(f"Max Doppler: {urban_data['metadata']['fd_max']:.2f} Hz")

FileNotFoundError: [Errno 2] No such file or directory: 'D:/AIT/deepmimodata/processed/test_dataset.pkl'

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class TemporalCSIDataset(Dataset):
    def __init__(self, X, Y):
        # Convert complex to real: stack [real, imag] as channels
        self.X = np.stack([X.real, X.imag], axis=-1)  # (N,T,Nr,Nt,Nk,2)
        self.Y = np.stack([Y.real, Y.imag], axis=-1)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return torch.FloatTensor(self.X[idx]), torch.FloatTensor(self.Y[idx])

# Create dataset
dataset = TemporalCSIDataset(X_train, Y_train)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Use in training
for X_batch, Y_batch in loader:
    # X_batch: (batch, T, Nr, Nt, Nk, 2)
    predictions = model(X_batch)
    loss = criterion(predictions, Y_batch)

In [ ]:
import matplotlib.pyplot as plt

# Get one sequence
seq = X_train[0, :, 0, 0, 0]  # First sequence, first antenna/subcarrier

# Plot magnitude and phase
time_ms = np.arange(len(seq)) * 0.01  # Assuming 10 μs sampling

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(time_ms, np.abs(seq))
plt.xlabel('Time (ms)')
plt.ylabel('|H(t)|')
plt.title('Channel Magnitude')

plt.subplot(1, 2, 2)
plt.plot(time_ms, np.angle(seq))
plt.xlabel('Time (ms)')
plt.ylabel('Phase (rad)')
plt.title('Doppler Phase Shift')
plt.tight_layout()
plt.show()